In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.chdir("/kaggle/input/competitive-data-science-predict-future-sales/")
files=glob.glob('./*.csv')
files
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_set = pd.read_csv(files[3])
test_set = pd.read_csv(files[5])
items_data = pd.read_csv(files[0])
item_categories_data = pd.read_csv(files[2])
submit_data = pd.read_csv(files[1])
shops_data = pd.read_csv(files[4])

In [ ]:
print("shape of train:",train_set.shape)
print("shape of test:",test_set.shape)
print("shape of submission:",submit_data.shape)
print("shape of items:",items_data.shape)
print("shape of item_cats:",item_categories_data.shape)
print("shape of shops:",shops_data.shape)

In [ ]:
print("column count of train:",len(train_set.axes[1]))
print("column count of test:",len(test_set.axes[1]))
print("column count of submission:",len(submit_data.axes[1]))
print("column count of items:",len(items_data.axes[1]))
print("column count of item_cats:",len(item_categories_data.axes[1]))
print("column count of shops:",len(shops_data.axes[1]))


In [ ]:
train_set=pd.merge(train_set,shops_data,how='left',on='shop_id')
train_set=pd.merge(train_set,items_data,how='left',on='item_id')
train_set=pd.merge(train_set,item_categories_data,how='left',on='item_category_id')

In [ ]:
del train_set["shop_id"]
del train_set["item_id"]
del train_set["item_category_id"]

In [ ]:
train_set

In [ ]:
sales_monthly=train_set.groupby(["date_block_num"])["item_cnt_day"].sum()

In [ ]:
sales_monthly.plot()
plt.show()

In [ ]:
test_set=pd.merge(test_set,shops_data,how='left',on='shop_id')
test_set=pd.merge(test_set,items_data,how='left',on='item_id')
test_set=pd.merge(test_set,item_categories_data,how='left',on='item_category_id')

In [ ]:
del test_set["shop_id"]
del test_set["item_id"]
del test_set["item_category_id"]
del test_set["ID"]

In [ ]:
test_set

In [ ]:
train_set['date'] = pd.to_datetime(train_set.date, format='%d.%m.%Y')
train_set.index = train_set.date

In [ ]:
train_set['item_cnt_day'].interpolate(method='linear', inplace=True)

In [ ]:
fig = plt.figure(figsize=(15,8))
train_set.item_cnt_day.plot()
plt.legend(loc='best')
plt.title('Daily Count', fontsize=14)
plt.show()

In [ ]:
#Ml Model
import fbprophet

df = train_set[['date','item_cnt_day']]

 
df = df.rename(columns={'date': 'ds', 'item_cnt_day': 'y'})
 
df['ds'] = pd.to_datetime(df['ds'], format='%d.%m.%Y')
 

In [ ]:
df = df[(df['ds']>='2015-09-01')&(df['ds']<='2015-10-01')]
 
df = df.sort_values(by=['ds'],ascending=True)
 
plt.plot(df['ds'], df['y'],'r')

plt.draw()
 
df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.15)
df_prophet.fit(df)
 
countofpre=30
df_forecast = df_prophet.make_future_dataframe(periods= countofpre, freq='D')
 
df_forecast = df_prophet.predict(df_forecast)
 
df_prophet.plot(df_forecast, xlabel = 'Date', ylabel = 'Count')

 
plt.show()